<h1 style="text-align: center">Тестовое задание по графовым базам данных</h1>

1) Установить 2 графовые базы из списка 

    Предпочтительные - neo4j, nebula, arangodb
    Предпочтительный язык запросов cypher

2) Создать ipynb ноутбук в котором:
3) Считать данные из источника https://docs.yandex.ru/docs/view?url=ya-disk-public%3A%2F%2FUgFAtb%2FEhHr%2BXAD9459IU56xcdad4cPrFCfTNup7PdJqr4sHmC%2FKML8Kb%2Byd3609q%2FJ6bpmRyOJonT3VoXnDag%3D%3D%3A%2Fdata_test.csv&name=data_test.csv
4) Внести данные из таблицы в графовую БД
5) Построить графовое представление в БД, осуществить несколько запросов на языке запросов к графовой БД
6) Найти взаимосвязи визуально и с помощью алгоритмов (алгоритмы на ваше усмотрение)
7) Написать rest сервис на python к графовой БД в котором на вход поступает ФИО, на выходе graphml или json
8) Результаты представить на гитхаб и в виде кода + небольшой презентации
9) Прислать ссылку на решение и резюме в телеграм @frankshikhaliev
10) Также надо будет заполнить форму

    Срок выполнения задания - около 10 дней, если вы не успеваете можете взять больше времени

<hr>



In [1]:
import pandas as pd
from py2neo import Graph

1. Используемая СУБД: neo4j. Управление СУБД осуществляется через библиотеку py2neo
2. Ноутбук создан на базе Jupyter Notebook. Я использовал расширение для vscode. Однако представление этого ноутбука можно вывести и через браузер
3. Данные из csv были скачаны и помещены в локальное хранилище. На этом этапе постановка задачи предполагает несколько решений: 
    1) скачать файл 
    2) обратиться через метод read_csv библиотеки pandas 
    3) использовать команду на языке запросов Cypher: LOAD CSV FROM и подобные конструкции. 
    
    Я выбрал первый путь.


In [2]:
df = pd.read_csv('data_test.csv', delimiter=';') # Считываем csv файл, в качестве разделителя используем точку с запятой

In [3]:
graph = Graph("http://localhost:7474/db/data/", user="neo4j", password="31337") # Подключение к БД neo4j
graph.delete_all() # Превентивно очищаем БД. Данная конструкция скорее для подстраховки при неоднократном запуске

4. Запись данных в БД происходит ниже. Механизм такой: данные были трансформированы в список, который содержит три элемента id, ФИО первого участника, ФИО второго участника. Необходимо построчно извлечь каждый столбец с данными. Запись можно было бы сделать двумя путями: 
    1) через экземпляр классов Node, Relationship 
    2) С использованием синтаксиса языка запросов Cypher

Я выбрал второй вариант, поскольку в задании упор делается на Cypher
Конструкция запроса следующая: команда MERGE отвечает за слияние элементов, создаются переменные, p1 и p2, ярлык переменных - Person, переменные включает в себя имена ФИО первого и второго участника

Далее происходит установление взаимосвязи (изначально планировалось установить в качестве ярлыка связи id, но это вызывает ошибку)

Последней командой мы передаем наш запрос в базу.

In [4]:
data_list = df.values.tolist()

for record in data_list:
    persons_relationship = str(record[0])
    person1 = record[1] 
    person2 = record[2]
    query = "MERGE (p1:Person {name: '" + person1 + "'}) MERGE (p2:Person {name: '" + person2 + "'}) MERGE (p1)-[:CONNECTED]->(p2)"
    graph.run(query)


<h3></h3>

5. Итак, графовое представление создано, мы можем убедится в этом либо перейдя в браузере по адресу:
http://localhost:7474/browser/ 

https://imgur.com/a/cD0Dq54


либо делая запросы в ноутбуке




5.1 Далее показана реализация нескольких запросов 

In [5]:
graph.run('MATCH (n:Person) RETURN n.person LIMIT 250') # Вывод списка имен

n.person
null
null
null


In [6]:
graph.run('MATCH (p:Person) WITH p, size((p)--()) as rel_count WHERE rel_count > 1  RETURN p.name,rel_count LIMIT 10000') # Вывод нод имеющих более одной связи

p.name,rel_count
Ахромеева Алина Ивановна,50
Башнина Антонина Глебовна,14
Медведева Дарья Алексеевна,6


In [7]:
graph.run("MATCH (n:Person) WHERE n.name CONTAINS ' Александр '  RETURN n.name") # Поиск по регулярным выражением

n.name
Сероштанов Александр Евгеньевич
Дальский Александр Егорович
Крюковский Александр Олегович


6. Анализируя датасет методом df.count() можно заметить что количество строк равно 5000
При этом около 70 значений в первом списке людей, и 6 значений во втором списке людей являются дубликатами, то есть повторяющимися. Это означает что одно человек может иметь взаимосвязь с несколькими людьми.

При добавлении в базу визуально и в целом этого не видно, поскольку цепочка человек-человек разбивается попарно, но в то же время согласно условиям задачи необходимо вычленить их. В целом вычленения одного элемента имеющего несколько связей уже было реализовано на одном из тестовых запросов выше.
Однако нам необходимо промаркировать эту выборку. Для чего был использован инструмент SET и был установлен красный цвет (в отличии от дефолтного зелёного).

https://imgur.com/a/4iYPeqR

In [8]:
df.count()

id события                 5000
ФИО участника события 1    5000
ФИО участника события 2    5000
dtype: int64

In [9]:
df['ФИО участника события 1'].duplicated().value_counts(), df['ФИО участника события 2'].duplicated().value_counts()

(False    4930
 True       70
 Name: ФИО участника события 1, dtype: int64,
 False    4994
 True        6
 Name: ФИО участника события 2, dtype: int64)

In [10]:
query = "MATCH (p:Person) WITH p, size((p)--()) as rel_count WHERE rel_count > 1 SET p:RedPerson"
graph.run(query)

(No data)